In [5]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('DISTANCE_MATRIX_API_KEY')

def fetch_and_append_distances(input_file, API_KEY):
    # Load the input data
    sample_data = pd.read_csv(input_file)

    # Temporary file to store updated data
    temp_file = input_file + '.tmp'

    # Open the temporary file for writing
    with open(temp_file, 'w') as f_out:
        # Write the header with the new 'travel_time' column
        f_out.write(','.join(sample_data.columns.tolist() + ['travel_time']) + '\n')

        # Iterate through each row in the sample data
        for index, row in sample_data.iterrows():
            origin = row['origin']
            destination = row['destination']
            
            # API request
            url = f"https://api.distancematrix.ai/maps/api/distancematrix/json?origins={origin}&destinations={destination}&key={API_KEY}"
            response = requests.get(url)
            
            # Print the response for debugging
            print(f"Response for {origin} to {destination}: {response.text}")

            # Default value for travel time in case of an error
            travel_time = 'N/A'

            if response.status_code == 200:
                data = response.json()
                if data['status'] == 'OK':
                    distance_info = data['rows'][0]['elements'][0]
                    if distance_info['status'] == 'OK':
                        travel_time = distance_info['duration']['text']

            # Append the updated row to the temporary file
            updated_row = row.tolist() + [travel_time]
            f_out.write(','.join(map(str, updated_row)) + '\n')

    # Replace the original file with the updated temporary file
    os.replace(temp_file, input_file)

# Set the path to your file and get your API key from environment variable
input_file = '/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance_sample.csv'

# Call the function to append travel time
fetch_and_append_distances(input_file, API_KEY)

print("Travel time data appended to CSV successfully.")

Response for 41.295893,-93.104232 to 41.687183,-93.772041: {"destination_addresses":["Grimes, Iowa, USA"],"origin_addresses":["Knoxville Township, Iowa 50138, USA"],"rows":[{"elements":[{"distance":{"text":"86.3 km","value":86335},"duration":{"text":"55 mins","value":3317},"origin":"41.295893,-93.104232","destination":"41.687183,-93.772041","status":"OK"}]}],"status":"OK"}
Response for 38.763463,-77.186063 to 38.960109,-78.187619: {"destination_addresses":["Baby, Front Royal, VA 22630, USA"],"origin_addresses":["7112 Lamar Dr, Springfield, VA 22150, USA"],"rows":[{"elements":[{"distance":{"text":"107.1 km","value":107099},"duration":{"text":"1 hour 9 mins","value":4177},"origin":"38.763463,-77.186063","destination":"38.960109,-78.187619","status":"OK"}]}],"status":"OK"}
Response for 44.12036,-69.24005 to 43.837601,-70.445011: {"destination_addresses":["30 Landing Rd, Windham, ME 04062, USA"],"origin_addresses":["350 Main St, Warren, ME 04864, USA"],"rows":[{"elements":[{"distance":{"te

In [14]:
# Function to convert travel time to minutes
def convert_to_minutes(time_str):
    if pd.isna(time_str) or 'min' not in time_str:
        return None
    time_parts = time_str.split()
    minutes = 0
    if 'hour' in time_str or 'hours' in time_str:
        minutes += int(time_parts[0]) * 60
        if len(time_parts) > 2:
            minutes += int(time_parts[2])
    else:
        minutes = int(time_parts[0])
    return minutes

# Process the CSV file
input_file = '/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance_sample.csv'
output_file = '/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/cleaned_time_distance_sample.csv'

with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
    first_line = True
    for line in f_in:
        parts = line.strip().split(',')
        # Skip header or incorrect lines
        if first_line or len(parts) < 7:
            first_line = False
            continue
        origin = f'"{parts[0]},{parts[1]}"'
        destination = f'"{parts[2]},{parts[3]}"'
        ruca = parts[4]
        distance = parts[5]
        travel_time = convert_to_minutes(parts[6])
        f_out.write(f'{origin},{destination},{ruca},{distance},{travel_time}\n')

print("Data cleaned and saved successfully.")

Data cleaned and saved successfully.


In [28]:
import pandas as pd
import json

# Function to convert kilometers to miles
def km_to_miles(km):
    return km * 0.621371

# Load the CSV data
csv_data = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/cleaned_time_distance_sample.csv')

# Initialize a dictionary to store distance data
distance_data = {}

# Read and parse the JSON file
with open('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/raw_api_output_distance_matrix.json', 'r') as file:
    for line in file:
        if 'Response for' in line and ':' in line:
            try:
                # Extract JSON part and response coordinates from the line
                response_prefix, json_str = line.split(': ', 1)
                json_data = json.loads(json_str)
                response_coords = response_prefix.split('Response for ')[1]
                origin_response, destination_response = response_coords.split(' to ')

                # Check if distance data is available
                if 'distance' in json_data['rows'][0]['elements'][0]:
                    distance_km = json_data['rows'][0]['elements'][0]['distance']['value'] / 1000
                    key = (origin_response, destination_response)
                    distance_data[key] = distance_km
            except json.JSONDecodeError:
                continue

# Check and create the 'distance' column if not exists
if 'distance' not in csv_data.columns:
    csv_data['distance'] = None

# Iterate over the CSV data and update distances
for index, row in csv_data.iterrows():
    origin_csv = row['origin']
    destination_csv = row['destination']
    csv_key = (origin_csv, destination_csv)

    if csv_key in distance_data:
        distance_miles = km_to_miles(distance_data[csv_key])
        csv_data.at[index, 'distance'] = distance_miles

# Save the updated CSV
csv_data.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance_sample.csv', index=False)